## Le cycle de vie

In [1]:
%%shell
(cd samples/hellojpa && mvn --quiet clean package )

In [72]:
%%shell
docker start H2-JPADemo || docker run --quiet --detach --rm --name=H2-JPADemo \
    -e PUID=$UID -e PGID=$GID \
    -e H2_ARGS="-tcp -tcpAllowOthers -ifNotExists" \
    -p 9092:9092 \
    -v h2-data:/opt/h2-data \
    brunoe/docker-database-h2:develop

H2-JPADemo


In [2]:
%jars samples/hellojpa/target/*-SNAPSHOT-jar-with-dependencies.jar

import jakarta.persistence.*;
import fr.univtln.bruno.demos.jpa.hello.DatabaseManager;
System.setProperty("jakarta.persistence.jdbc.url","jdbc:h2:tcp://host.docker.internal/hellojpa-db");

try (EntityManager entityManager = DatabaseManager.ENTITY_MANAGER_FACTORY.createEntityManager()) { }

18:19:25.501 INFO  f.u.b.d.jpa.hello.DatabaseManager - Override JPA config jakarta.persistence.jdbc.url with system property jakarta.persistence.jdbc.url
18:19:25.505 INFO  f.u.b.d.jpa.hello.DatabaseManager - Override JPA config jakarta.persistence.jdbc.user with system property jakarta.persistence.jdbc.user
18:19:25.506 INFO  f.u.b.d.jpa.hello.DatabaseManager - Override JPA config jakarta.persistence.jdbc.password with system property jakarta.persistence.jdbc.password
18:19:27.078 INFO  com.mchange.v2.log.MLog - MLog clients using slf4j logging.
18:19:27.309 INFO  SQL dialect - HHH000400: Using dialect: org.hibernate.dialect.H2Dialect


In [4]:
import org.slf4j.Logger;
import org.slf4j.LoggerFactory;

Logger log = LoggerFactory.getLogger("Notebook");

In [3]:
%%javasrcClassByName Customer
samples/hellojpa/src/main/java/fr/univtln/bruno/demos/jpa/hello/samples/ex_hook/Customer.java

```Java
@Entity
@Data
@RequiredArgsConstructor(staticName = "of")
@NoArgsConstructor(access = AccessLevel.PROTECTED)
@Table(name = "CUSTOMER", schema = "EX_HOOK")
@Slf4j
public class Customer {

    @Id
    @GeneratedValue
    private long id;

    @NonNull
    private String name;

    @PrePersist
    public void logPrePersist() {
        log.info("About to persist: " + this);
    }

    @PostPersist
    public void logPostPersist() {
        log.info("Persisted: " + this);
    }

    @PreUpdate
    public void logPPreUpdate() {
        log.info("About to update: " + this);
    }

    @PostUpdate
    public void logPostUpdate() {
        log.info("Removed: " + this);
    }

    @PreRemove
    public void logPPreRemove() {
        log.info("About to remove: " + this);
    }

    @PostRemove
    public void logPostRemove() {
        log.info("Removed: " + this);
    }

    @PostLoad
    public void logPostLoad() {
        log.info("Loaded: " + this);
    }
}
```

In [5]:
import fr.univtln.bruno.demos.jpa.hello.samples.ex_hook.Customer;

            try (EntityManager entityManager = DatabaseManager.ENTITY_MANAGER_FACTORY.createEntityManager()) {

                entityManager.getTransaction().begin();
                Customer customer = Customer.of("Peter");
                log.info(customer.toString());
                entityManager.persist(customer);
                customer.setName("Pierre");
                entityManager.getTransaction().commit();
                try (EntityManager anotherEM = DatabaseManager.ENTITY_MANAGER_FACTORY.createEntityManager()) {
                    log.info("FOUND " + anotherEM.find(Customer.class, 1L));
                }
                entityManager.getTransaction().begin();
                entityManager.remove(customer);
                entityManager.getTransaction().commit();
            }


18:19:28.770 INFO  Notebook - Customer(id=0, name=Peter)
18:19:28.777 INFO  f.u.b.d.j.h.samples.ex_hook.Customer - About to persist: Customer(id=0, name=Peter)
18:19:28.855 INFO  f.u.b.d.j.h.samples.ex_hook.Customer - About to update: Customer(id=1, name=Pierre)
18:19:28.867 INFO  f.u.b.d.j.h.samples.ex_hook.Customer - Persisted: Customer(id=1, name=Pierre)
18:19:28.872 INFO  f.u.b.d.j.h.samples.ex_hook.Customer - Removed: Customer(id=1, name=Pierre)
18:19:28.904 INFO  f.u.b.d.j.h.samples.ex_hook.Customer - Loaded: Customer(id=1, name=Pierre)
18:19:28.911 INFO  Notebook - FOUND Customer(id=1, name=Pierre)
18:19:28.914 INFO  f.u.b.d.j.h.samples.ex_hook.Customer - About to remove: Customer(id=1, name=Pierre)
18:19:28.919 INFO  f.u.b.d.j.h.samples.ex_hook.Customer - Removed: Customer(id=1, name=Pierre)


In [6]:
%%javasrcClassByName EntityMonitorListener
samples/hellojpa/src/main/java/fr/univtln/bruno/demos/jpa/hello/samples/ex_hook/EntityMonitorListener.java

```Java
@Slf4j
public class EntityMonitorListener {

    @PrePersist
    @PreUpdate
    @PreRemove
    private void beforeAnyUpdate(Object entity) {
        log.info("-->about to change in DB: " + entity);
    }

    @PostPersist
    @PostUpdate
    @PostRemove
    private void afterAnyUpdate(Object entity) {
        log.info("--> changed in DB" + entity);
    }

    @PostLoad
    private void afterLoad(Object entity) {
        log.info("--> loaded from DB: " + entity);
    }
}
```

In [7]:
%%javasrcClassByName CustomerWithListener
samples/hellojpa/src/main/java/fr/univtln/bruno/demos/jpa/hello/samples/ex_hook/CustomerWithListener.java

```Java
@Entity
@Data
@RequiredArgsConstructor(staticName = "of")
@NoArgsConstructor(access = AccessLevel.PROTECTED)
@Table(name = "CUSTOMER_BIS")
@Log
@EntityListeners(EntityMonitorListener.class)
public class CustomerWithListener {

    @Id
    @GeneratedValue
    private long id;

    @NonNull
    private String name;
}
```

In [8]:
import fr.univtln.bruno.demos.jpa.hello.samples.ex_hook.CustomerWithListener;

try (EntityManager entityManager = DatabaseManager.ENTITY_MANAGER_FACTORY.createEntityManager()) {

                entityManager.getTransaction().begin();
                CustomerWithListener customer = CustomerWithListener.of("Peter");
                log.info(customer.toString());
                entityManager.persist(customer);
                customer.setName("Pierre");
                entityManager.getTransaction().commit();

                try (EntityManager anotherEM = DatabaseManager.ENTITY_MANAGER_FACTORY.createEntityManager()) {
                    log.info("FOUND " + anotherEM.find(CustomerWithListener.class, 1L));
                }

                entityManager.getTransaction().begin();
                entityManager.remove(customer);
                entityManager.getTransaction().commit();
            }        

18:19:29.532 INFO  Notebook - CustomerWithListener(id=0, name=Peter)
18:19:29.533 INFO  f.u.b.d.j.h.s.e.EntityMonitorListener - -->about to change in DB: CustomerWithListener(id=0, name=Peter)
18:19:29.537 INFO  f.u.b.d.j.h.s.e.EntityMonitorListener - -->about to change in DB: CustomerWithListener(id=1, name=Pierre)
18:19:29.539 INFO  f.u.b.d.j.h.s.e.EntityMonitorListener - --> changed in DBCustomerWithListener(id=1, name=Pierre)
18:19:29.542 INFO  f.u.b.d.j.h.s.e.EntityMonitorListener - --> changed in DBCustomerWithListener(id=1, name=Pierre)
18:19:29.550 INFO  f.u.b.d.j.h.s.e.EntityMonitorListener - --> loaded from DB: CustomerWithListener(id=1, name=Pierre)
18:19:29.552 INFO  Notebook - FOUND CustomerWithListener(id=1, name=Pierre)
18:19:29.554 INFO  f.u.b.d.j.h.s.e.EntityMonitorListener - -->about to change in DB: CustomerWithListener(id=1, name=Pierre)
18:19:29.557 INFO  f.u.b.d.j.h.s.e.EntityMonitorListener - --> changed in DBCustomerWithListener(id=1, name=Pierre)
